<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/Tweet_push.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import tweepy

# class MyStreamListener(tweepy.StreamListener):

#    def on_status(self, status):
#        print(status.text)

#    def on_error(self, status_code):
#        if status_code == 420:
#            #returning False in on_data disconnects the stream
#            return False

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth)

# myStreamListener = MyStreamListener()
# myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [ ]:
# myStream.filter(languages=['english'])

In [3]:
!pip -q install pycurl oauth2 kafka-python

In [4]:
import time
import pycurl
import urllib
import json
import oauth2 as oauth

In [5]:
consumer_key  = "2wrYTQnh4poBbF5MsubR3sk4k"
consumer_secret = "93Yi0isNNgNPW4bFo8hsMpANUIw61lofgRRpz8H39P5V6o63tl"
access_token  = "2761651876-LoLkgE3EFYkqIHSGEPvzB3udtmATj4XNem2MemW"
access_token_secret = "1uDlUpdk6mrinFKFl3vN06xIVeORaa60EjubXrEOSpYKc"

In [6]:
API_ENDPOINT_URL = 'https://stream.twitter.com/1.1/statuses/filter.json'
USER_AGENT = 'TwitterStream 1.0' # This can be anything really

In [7]:
# You need to replace these with your own values
OAUTH_KEYS = {'consumer_key': consumer_key,
              'consumer_secret': consumer_secret,
              'access_token_key': access_token,
              'access_token_secret': access_token_secret}
print(OAUTH_KEYS)

{'consumer_key': '2wrYTQnh4poBbF5MsubR3sk4k', 'consumer_secret': '93Yi0isNNgNPW4bFo8hsMpANUIw61lofgRRpz8H39P5V6o63tl', 'access_token_key': '2761651876-LoLkgE3EFYkqIHSGEPvzB3udtmATj4XNem2MemW', 'access_token_secret': '1uDlUpdk6mrinFKFl3vN06xIVeORaa60EjubXrEOSpYKc'}


In [8]:
POST_PARAMS = {'include_entities': 0,
               'stall_warning': 'true',
            #  'track': '#',
               'locations': '-171, 18, -66, 71'}

In [9]:
class TwitterStream:
    def __init__(self, timeout=False):
        self.oauth_token = oauth.Token(key=OAUTH_KEYS['access_token_key'], secret=OAUTH_KEYS['access_token_secret'])
        self.oauth_consumer = oauth.Consumer(key=OAUTH_KEYS['consumer_key'], secret=OAUTH_KEYS['consumer_secret'])
        self.conn = None
        self.buffer = ''
        self.timeout = timeout
        self.setup_connection()

    def setup_connection(self):
        """ Create persistant HTTP connection to Streaming API endpoint using cURL.
        """
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        # Restart connection if less than 1 byte/s is received during "timeout" seconds
        if isinstance(self.timeout, int):
            self.conn.setopt(pycurl.LOW_SPEED_LIMIT, 1)
            self.conn.setopt(pycurl.LOW_SPEED_TIME, self.timeout)
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL)
        self.conn.setopt(pycurl.USERAGENT, USER_AGENT)
        # Using gzip is optional but saves us bandwidth.
        self.conn.setopt(pycurl.ENCODING, 'deflate, gzip')
        self.conn.setopt(pycurl.POST, 1)
        self.conn.setopt(pycurl.POSTFIELDS, urllib.urlencode(POST_PARAMS))
        self.conn.setopt(pycurl.HTTPHEADER, ['Host: stream.twitter.com',
                                             'Authorization: %s' % self.get_oauth_header()])
        # self.handle_tweet is the method that are called when new tweets arrive
        self.conn.setopt(pycurl.WRITEFUNCTION, self.handle_tweet)

    def get_oauth_header(self):
        """ Create and return OAuth header.
        """
        params = {'oauth_version': '1.0',
                  'oauth_nonce': oauth.generate_nonce(),
                  'oauth_timestamp': str(int(time.time()))}
        req = oauth.Request(method='POST', parameters=params, url='%s?%s' % (API_ENDPOINT_URL,
                                                                             urllib.urlencode(POST_PARAMS)))
        req.sign_request(oauth.SignatureMethod_HMAC_SHA1(), self.oauth_consumer, self.oauth_token)
        
       
        return str(req.to_header()['Authorization']).encode('utf-8')

    def start(self):
        """ Start listening to Streaming endpoint.
        Handle exceptions according to Twitter's recommendations.
        """
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                # Network error, use linear back off up to 16 seconds
                print('Network error: %s' % self.conn.errstr())
                print('Waiting %s seconds before trying again' % backoff_network_error)
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)
            if sc == 420:
                # Rate limit, use exponential back off starting with 1 minute and double each attempt
                print('Rate limit, waiting %s seconds' % backoff_rate_limit)
                time.sleep(backoff_rate_limit)
                backoff_rate_limit *= 2
            else:
                # HTTP error, use exponential back off up to 320 seconds
                print('HTTP error %s, %s' % (sc, self.conn.errstr()))
                print('Waiting %s seconds' % backoff_http_error)
                time.sleep(backoff_http_error)
                backoff_http_error = min(backoff_http_error * 2, 320)

    def handle_tweet(self, data):
        """ This method is called when data is received through Streaming endpoint.
        """
        self.buffer += str(data)
        if data.endswith(b'\r\n') and self.buffer.strip():
            # complete message received
            message = json.loads(self.buffer)
            self.buffer = ''
            msg = ''
            if message.get('limit'):
                print('Rate limiting caused us to miss %s tweets' % (message['limit'].get('track')))
            elif message.get('disconnect'):
                raise Exception('Got disconnect: %s' % message['disconnect'].get('reason'))
            elif message.get('warning'):
                print('Got warning: %s' % message['warning'].get('message'))
            else:
                print(message)
                print('Got tweet with text: %s' % message.get('text'))
            
            producer.send('tweetv1', value=message) 

In [10]:
from kafka import KafkaProducer
from kafka.errors import KafkaError
from json import dumps

AnalysisException: 'Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".;'

In [ ]:
producer = KafkaProducer(bootstrap_servers=['18.216.34.97:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [ ]:
ts = TwitterStream()
ts.setup_connection()
ts.start()

In [ ]:
producer.close()

In [1]:
from kafka import KafkaConsumer
from json import loads

In [2]:
consumer = KafkaConsumer(
    'numtest',
     bootstrap_servers=['18.216.34.97:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group310',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

NoBrokersAvailable: NoBrokersAvailable

In [ ]:
for message in consumer:
    message = message.value
    print(message)
    print(type(message))